# CREACIÓN DEL DATAFRAME

Este cuaderno sigue los siguientes pasos:

1. Comienza importando las bibliotecas necesarias para el análisis y manipulación de los datos.
2. Define constantes importantes para el análisis, incluyendo los nombres de los meses y los encabezados que se utilizarán en el DataFrame.
3. Se definen varias funciones útiles para el proceso de creación del DataFrame:
* `numero_a_mes`: Esta función transforma un número entero entre 1 y 12 en el nombre del mes correspondiente.
* `detectar_fila_cabecera`: Dado que la ubicación de la cabecera puede variar entre las hojas de los archivos Excel, esta función se encarga de identificar la fila correcta de la cabecera en cada hoja.
* `dataframe_mes` Esta función se encarga de generar un DataFrame específico para cada mes.
* `crear_dataframe`: Esta es la función principal que utiliza las funciones auxiliares anteriormente descritas para generar los DataFrames.
4. La función `crear_dataframe` itera sobre un rango de años y meses. Para cada combinación de mes y año, busca el archivo de Excel correspondiente y crea un DataFrame con los datos del mes específico.
5. Al final, se crean dos DataFrames: `dataframe`, que contiene los datos sin relleno manual, y `dataframe_edit`, que contiene los datos con relleno manual. Ambos DataFrames se guardan en archivos CSV para su posterior análisis.

In [1]:
# Importación de bibliotecas necesarias
from calendar import monthrange
import os.path
import numpy as np
import pandas as pd
from datetime import datetime, date
import xlrd 
from openpyxl import load_workbook

In [2]:
# Definición de constantes
MESES = {
        1: 'Enero',
        2: 'Febrero',
        3: 'Marzo',
        4: 'Abril',
        5: 'Mayo',
        6: 'Junio',
        7: 'Julio',
        8: 'Agosto',
        9: 'Septiembre',
        10: 'Octubre',
        11: 'Noviembre',
        12: 'Diciembre'
        }
HEADERS = [ ('FECHA', ''),
            ('HORA', ''),
            ('CAUDAL','Entr l/s'),
            ('Cal 1ria', 'Kg.'),
            ('Cal 1ria', 'Dosis'),
            ('AGUA CRUDA', 'p.H'),
            ('AGUA CRUDA', 'Color'),
            ('AGUA CRUDA', 'NTU'),
            ('AGUA CRUDA', 'Alcalinidad'),
            ('AGUA CRUDA', 'Conductividad'),
            ('COAGULANTE', 'Granulado'),
            ('COAGULANTE', 'Liquido'),
            ('COAGULANTE', 'Dosis'),
            ('AGUA TRATADA', 'Cloro'),
            ('AGUA TRATADA', 'Alcalinidad'),
            ('AGUA TRATADA', 'p.H'),
            ('AGUA TRATADA', 'Color'),
            ('AGUA TRATADA', 'NTU')
          ]

In [3]:
# Creación de MultiIndex para el encabezado de los DataFrame
header = pd.MultiIndex.from_tuples(HEADERS)

In [4]:
# Inicialización de DataFrames vacíos
df = pd.DataFrame()
df_total = pd.DataFrame()

In [ ]:
# Definición de rangos para años, meses y horas
años = list(range(2013,2023))
meses = list(range(1,13))
horas = list(range(25))

In [5]:
# Función para convertir un número de mes en el nombre del mes
def numero_a_mes(numero):
    if 1 <= numero <= 12:
        return MESES[numero]
    else:
        raise ValueError("El número debe estar entre 1 y 12.")

In [6]:
# Función para detectar la fila de la cabecera en un archivo de Excel
def detectar_fila_cabecera(archivo_excel):
    wb = load_workbook(filename=archivo_excel, read_only=True)
    filas_cabecera = {}
    palabra_clave = 'HO'

    for hoja in wb:
        fila_cabecera = None
        for index, row in enumerate(hoja.iter_rows(), start=1):
            if any(palabra_clave in str(cell.value) for cell in row if cell.value is not None):
                fila_cabecera = index
                break
        filas_cabecera[hoja.title] = fila_cabecera

    return filas_cabecera

In [7]:
# Función para crear un DataFrame para un mes específico
def dataframe_mes(fila, mes, days, año, archivo):
    for i in list(range(days)):
        x = f'{i+1}'
        fecha = f'{i+1}'+'/'+f'{mes}'+'/'+f'{año}'
        if i == 0:
            df_empty = pd.DataFrame(columns=HEADERS)
            df = pd.read_excel(archivo, sheet_name=x, header=[0, 1], skiprows=fila[x] - 1, nrows=25)
            common_columns = df_empty.columns.intersection(df.columns)
            df_common = df[common_columns]
            df = pd.concat([df_empty, df_common], axis=0, ignore_index=True)
            df[('FECHA', '')] = fecha
            df[('HORA', '')] = horas
        else:
            df_read = pd.read_excel(archivo, sheet_name=x, header=[0, 1], skiprows=fila[x] - 1, nrows=25)
            df_read[('FECHA', '')] = fecha
            df_read[('HORA', '')] = horas
            df = pd.concat([df, df_read])

    return df

In [8]:
# Función principal para crear el DataFrame
def crear_dataframe(folder):

    for año in años:
        for mes in meses:
            CURR_NUM_DAYS = monthrange(año, mes)[1]
            mes_name = numero_a_mes(mes)
            archivo = f"../{folder}/{año}/{mes_name}{año}.xlsx"
            print(archivo)

            if os.path.exists(archivo):
                fila = detectar_fila_cabecera(archivo)
                df_mes = dataframe_mes(fila, mes, CURR_NUM_DAYS, año, archivo)
            else:
                print(f"Archivo no encontrado: {archivo}. Creando datos NaN para el mes {mes} del año {año}.")
                if mes == 12:
                    año = año+1
                    mes = 1
                else:
                    mes = mes+1

            if año == 2013 and mes == 1:
                df_total = df_mes
            else:
                df_total = pd.concat([df_total, df_mes])

    return df_total[HEADERS].reset_index(drop=True)

In [9]:
# Creación del DataFrame con los datos sin relleno manual
dataframe = crear_dataframe('data')
dataframe.to_csv('../data/dataframe.csv', sep=',')

../data/2013/Enero2013.xlsx
../data/2013/Febrero2013.xlsx
../data/2013/Marzo2013.xlsx
../data/2013/Abril2013.xlsx
../data/2013/Mayo2013.xlsx
../data/2013/Junio2013.xlsx
../data/2013/Julio2013.xlsx
../data/2013/Agosto2013.xlsx
../data/2013/Septiembre2013.xlsx
../data/2013/Octubre2013.xlsx
../data/2013/Noviembre2013.xlsx
../data/2013/Diciembre2013.xlsx
../data/2014/Enero2014.xlsx
../data/2014/Febrero2014.xlsx
../data/2014/Marzo2014.xlsx
../data/2014/Abril2014.xlsx
../data/2014/Mayo2014.xlsx
../data/2014/Junio2014.xlsx
../data/2014/Julio2014.xlsx
../data/2014/Agosto2014.xlsx
../data/2014/Septiembre2014.xlsx
../data/2014/Octubre2014.xlsx
../data/2014/Noviembre2014.xlsx
../data/2014/Diciembre2014.xlsx
../data/2015/Enero2015.xlsx
../data/2015/Febrero2015.xlsx
../data/2015/Marzo2015.xlsx
../data/2015/Abril2015.xlsx
../data/2015/Mayo2015.xlsx
../data/2015/Junio2015.xlsx
../data/2015/Julio2015.xlsx
../data/2015/Agosto2015.xlsx
../data/2015/Septiembre2015.xlsx
../data/2015/Octubre2015.xlsx
../da

In [ ]:
# Creación del DataFrame con los datos de relleno manual
dataframe_edit = crear_dataframe('data_edit')
dataframe_edit.to_csv('../data_edit/dataframe_edit.csv', sep=',')